# Интерактивная карта промышленности Москвы

In [15]:
import folium
from folium import plugins
import pandas as pd
import geopandas as gpd
import json
import re
import html
from shapely.geometry import Point
import osmnx as ox

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## Загрузка данных

In [16]:
# Предприятия (данные от data.mos.ru)
df = pd.read_csv('data/data-2601-2024-09-06.csv', sep=';', encoding='utf-8')
coords = df['geoData'].str.extract(r'coordinates=\[([0-9.-]+),\s*([0-9.-]+)\]')
df['lon'] = pd.to_numeric(coords[0])
df['lat'] = pd.to_numeric(coords[1])
df = df.dropna(subset=['lon', 'lat'])

In [17]:
# Административные округа из geojson
districts = gpd.read_file('data/moscow_districts.geojson')
okrugs = districts.dissolve(by='NAME_AO', as_index=False)
okrugs = okrugs[['NAME_AO', 'geometry']]
print(f"Загружено {len(okrugs)} административных округов")

Загружено 12 административных округов


In [18]:
# Берем промышленные зоны из OSM
industrial_zones = ox.features_from_place(
    "Moscow, Russia",
    tags={"landuse": "industrial"}
)
industrial_zones = industrial_zones.to_crs("EPSG:4326")
industrial_zones['area_km2'] = industrial_zones.to_crs("EPSG:32637").area / 1000000
print(f"Загружено {len(industrial_zones)} промзон")

Загружено 2483 промзон


In [19]:
#жд инфраструктура из osm
railway_lines = ox.features_from_place(
    "Moscow, Russia",
    tags={"railway": ["rail", "light_rail"]}
)
railway_lines = railway_lines.to_crs("EPSG:4326")
print(f"Загружено {len(railway_lines)} ж/д линий")

railway_stations = ox.features_from_place(
    "Moscow, Russia",
    tags={"railway": ["station", "halt", "platform"],
          "building": "train_station",
          "landuse": "railway"}
)
railway_stations = railway_stations.to_crs("EPSG:4326")
railway_stations = railway_stations[railway_stations.geometry.type.isin(['Point', 'Polygon', 'MultiPolygon'])]

if 'station' in railway_stations.columns:
    railway_stations = railway_stations[~railway_stations['station'].isin(['subway', 'tram', 'light_rail'])]

# не хотим показывать трамваи и метро фильтруем
if 'subway' in railway_stations.columns:
    railway_stations = railway_stations[railway_stations['subway'] != 'yes']
if 'tram' in railway_stations.columns:
    railway_stations = railway_stations[railway_stations['tram'] != 'yes']

print(f"Загружено {len(railway_stations)} ж/д станций и терминалов")

Загружено 6920 ж/д линий
Загружено 1048 ж/д станций и терминалов


## Анализ данных
Считаем статистику для округов

In [20]:
enterprises_gdf = gpd.GeoDataFrame(
    df, 
    geometry=gpd.points_from_xy(df.lon, df.lat),
    crs="EPSG:4326"
)

okrug_stats = {}
for idx, okrug in okrugs.iterrows():
    okrug_name = okrug['NAME_AO']

    #кол-во промышленных предприятий 
    enterprises_in_okrug = enterprises_gdf[enterprises_gdf.within(okrug.geometry)]
    enterprise_count = len(enterprises_in_okrug)

    #площадь промышленных зон 
    zones_in_okrug = industrial_zones[industrial_zones.within(okrug.geometry)]
    industrial_area = zones_in_okrug['area_km2'].sum()
    
    okrug_stats[okrug_name] = {
        'enterprises': enterprise_count,
        'industrial_area': industrial_area
    }
    
    okrugs.loc[idx, 'enterprises'] = enterprise_count
    okrugs.loc[idx, 'industrial_area'] = industrial_area
    
    print(f"{okrug_name}: {enterprise_count} предприятий, {industrial_area:.1f} км2 промзон")

Восточный: 118 предприятий, 11.3 км² промзон
Западный: 54 предприятий, 14.3 км² промзон
Зеленоградский: 50 предприятий, 3.3 км² промзон
Новомосковский: 28 предприятий, 12.8 км² промзон
Северный: 109 предприятий, 12.8 км² промзон
Северо-Восточный: 85 предприятий, 11.0 км² промзон
Северо-Западный: 31 предприятий, 6.1 км² промзон
Троицкий: 26 предприятий, 10.3 км² промзон
Центральный: 83 предприятий, 1.9 км² промзон
Юго-Восточный: 107 предприятий, 20.9 км² промзон
Юго-Западный: 54 предприятий, 5.4 км² промзон
Южный: 93 предприятий, 14.8 км² промзон


## Создание карты

In [21]:
m = folium.Map(
    location=[55.7558, 37.6173],
    zoom_start=10,
    tiles='CartoDB positron'
)

In [22]:
# Слой 1: Административные границы округов
folium.Choropleth(
    geo_data=okrugs,
    name='Административные округа',
    data=okrugs,
    columns=['NAME_AO', 'enterprises'],
    key_on='feature.properties.NAME_AO',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Количество промышленных предприятий',
    bins=5,
    highlight=True
).add_to(m)

folium.features.GeoJson(
    okrugs,
    name='Информация об округах',
    style_function=lambda x: {
        'fillColor': 'transparent',
        'color': 'transparent',
        'weight': 0,
        'fillOpacity': 0
    },
    tooltip=folium.features.GeoJsonTooltip(
        fields=['NAME_AO', 'enterprises', 'industrial_area'],
        aliases=['Округ:', 'Предприятий:', 'Промзоны (км2):'],
        labels=True,
        sticky=True,
        toLocaleString=True
    ),
    highlight_function=lambda x: {
        'weight': 3,
        'color': 'black',
        'fillOpacity': 0.1
    }
).add_to(m)

In [23]:
# Слой 2: Промышленные зоны
if not industrial_zones.empty:
    zone_features = []
    for idx, zone in industrial_zones.iterrows():
        zone_name = zone.get('name', f'Промзона №{idx}')
        zone_features.append({
            "type": "Feature",
            "properties": {
                "name": zone_name,
                "area": zone['area_km2']
            },
            "geometry": zone.geometry.__geo_interface__
        })
    
    zones_geojson = {
        "type": "FeatureCollection",
        "features": zone_features
    }
    
    folium.GeoJson(
        zones_geojson,
        name='Промышленные зоны',
        style_function=lambda x: {
            'fillColor': 'orange',
            'color': 'darkorange',
            'weight': 2,
            'fillOpacity': 0.3
        },
        tooltip=folium.GeoJsonTooltip(
            fields=['name', 'area'],
            aliases=['Название:', 'Площадь (км2):'],
            localize=True
        )
    ).add_to(m)

In [24]:
# Слой 3: Железнодорожная инфраструктура
if not railway_lines.empty:
    railway_feature_group = folium.FeatureGroup(name='Железнодорожная инфраструктура')

    railway_styles = {
        'rail': {'color': '#666666', 'weight': 3, 'opacity': 0.8},
        'light_rail': {'color': '#999999', 'weight': 2, 'opacity': 0.7},
    }

    for idx, line in railway_lines.iterrows():
        railway_type = line.get('railway', 'rail')
        style = railway_styles.get(railway_type, railway_styles['rail'])

        folium.GeoJson(
            line.geometry.__geo_interface__,
            style_function=lambda x, style=style: style
        ).add_to(railway_feature_group)

    if not railway_stations.empty:
        for idx, station in railway_stations.iterrows():
            station_name = html.escape(str(station.get('name', 'Ж/д станция')))

            if station.geometry.type == 'Point':
                folium.CircleMarker(
                    location=[station.geometry.y, station.geometry.x],
                    radius=3,
                    popup=station_name,
                    tooltip=station_name,
                    color='darkred',
                    fill=True,
                    fillColor='red',
                    fillOpacity=0.5
                ).add_to(railway_feature_group)
            else:
                terminal_name = html.escape(str(station.get('name', 'Ж/д терминал')))
                folium.GeoJson(
                    station.geometry.__geo_interface__,
                    style_function=lambda x: {
                        'fillColor': 'darkred',
                        'color': 'darkred',
                        'weight': 1,
                        'fillOpacity': 0.3
                    },
                    tooltip=terminal_name
                ).add_to(railway_feature_group)

    railway_feature_group.add_to(m)

/var/folders/ch/8921msk901z92snw0f1mhnk00000gn/T/ipykernel_2458/1580847494.py:23: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if station.geometry.type == 'Point':


In [25]:
# Слой 4: Промышленные предприятия
marker_cluster = plugins.MarkerCluster(name='Промышленные предприятия')

cat_colors = {
    'Пищевая промышленность': 'green',
    'Химическая промышленность': 'purple',
    'Радиоэлектроника и приборостроение': 'blue',
    'Машины и оборудование': 'orange',
    'Легкая промышленность': 'pink',
    'Фармацевтическая промышленность': 'red',
    'Металлургия и металлообработка': 'gray',
    'Топливно-энергетический комплекс': 'darkred'
}

# В данных из osm есть какая-то проблема, и нашел подсказку как можно очистить данные
def safe_text(text):
    if pd.isna(text):
        return "Не указано"
    text = str(text)
    text = ''.join(c for c in text if c.isalnum() or c in ' .,-()')
    return text[:100]

for _, row in df.iterrows():
    color = cat_colors.get(row.get('Category', ''), 'lightgray')
    name = safe_text(row.get('ShortName', 'Предприятие'))
    category = safe_text(row.get('Category', 'Категория'))

    folium.Marker(
        location=[row['lat'], row['lon']],
        tooltip=name,
        popup=folium.Popup(f"{name} - {category}", max_width=200),
        icon=folium.Icon(color=color, icon='industry', prefix='fa')
    ).add_to(marker_cluster)

marker_cluster.add_to(m)

In [26]:
# Слой 5: Тепловая карта
heat_data = df[['lat', 'lon']].values.tolist()
plugins.HeatMap(
    heat_data,
    name='Концентрация предприятий',
    min_opacity=0.2,
    max_zoom=11,
    radius=25,
    blur=15,
    gradient={
        0.0: 'blue',
        0.25: 'cyan',
        0.5: 'lime',
        0.75: 'yellow',
        1.0: 'red'
    }
).add_to(m)



Плагины на карту

In [27]:

folium.LayerControl(collapsed=False).add_to(m)
plugins.MousePosition(position='bottomleft', separator=' | ', prefix='Координаты:').add_to(m)
plugins.Fullscreen(position='topleft', title='Развернуть', title_cancel='Свернуть').add_to(m)
plugins.MiniMap(
    position='bottomright',
    width=150,
    height=150,
    collapsed_width=25,
    collapsed_height=25,
    zoom_level_offset=-5
).add_to(m)
plugins.MeasureControl(
    position='topleft',
    primary_length_unit='meters',
    secondary_length_unit='kilometers',
    primary_area_unit='sqmeters',
    secondary_area_unit='sqkilometers'
).add_to(m)
plugins.Geocoder().add_to(m)

In [28]:
# Статистика на карте
top3 = sorted(okrug_stats.items(), key=lambda x: x[1]['enterprises'], reverse=True)[:3]
total_industrial_area = int(sum([s['industrial_area'] for s in okrug_stats.values()]))

stat_html = '<div style="position: fixed; top: 280px; right: 20px; width: 300px; '
stat_html += 'background: white; z-index: 1000; font-size: 12px; '
stat_html += 'border: 2px solid #333; border-radius: 5px; padding: 15px; '
stat_html += 'box-shadow: 0 0 10px rgba(0,0,0,0.2);">'
stat_html += '<h3 style="margin: 0 0 10px 0;">Промышленность Москвы</h3>'
stat_html += '<b>Всего предприятий:</b> ' + str(len(df)) + '<br>'
stat_html += '<b>Административных округов:</b> ' + str(len(okrugs)) + '<br>'
stat_html += '<b>Промышленных зон:</b> ' + str(len(industrial_zones)) + '<br>'
stat_html += '<b>Общая площадь промзон:</b> ' + str(total_industrial_area) + ' км2<br>'
stat_html += '<b>Ж/д станций/терминалов:</b> ' + str(len(railway_stations)) + '<br>'
stat_html += '<hr>'
stat_html += '<b>Топ-3 округа по предприятиям:</b><br>'

for i, (okrug, stats) in enumerate(top3):
    stat_html += str(i+1) + '. ' + str(okrug) + ': ' + str(stats["enterprises"]) + '<br>'
stat_html += '<br/><br/><b>Автор карты:</b> Евгений Пастухов<br/>'
stat_html += '</div>'

m.get_root().html.add_child(folium.Element(stat_html))

m.save('index.html')
print("Карта сохранена")

Карта сохранена


## Итоговая статистика

In [29]:
top_okrugs = sorted(okrug_stats.items(), key=lambda x: x[1]['enterprises'], reverse=True)

print("Топ-5 округов по количеству предприятий:")
for i, (okrug, stats) in enumerate(top_okrugs[:5]):
    print(f"{i+1}. {okrug}: {stats['enterprises']} предприятий, {stats['industrial_area']:.1f} км2 промзон")

print(f"\nВсего предприятий: {len(df)}")
print(f"Общая площадь промзон: {sum([s['industrial_area'] for s in okrug_stats.values()]):.1f} км2")

print("\nТоп-5 категорий предприятий:")
category_counts = df['Category'].value_counts()
for i, (category, count) in enumerate(category_counts.head().items()):
    print(f"{i+1}. {category}: {count} предприятий")

Топ-5 округов по количеству предприятий:
1. Восточный: 118 предприятий, 11.3 км² промзон
2. Северный: 109 предприятий, 12.8 км² промзон
3. Юго-Восточный: 107 предприятий, 20.9 км² промзон
4. Южный: 93 предприятий, 14.8 км² промзон
5. Северо-Восточный: 85 предприятий, 11.0 км² промзон

Всего предприятий: 840
Общая площадь промзон: 125.0 км²

Топ-5 категорий предприятий:
1. Радиоэлектроника и приборостроение: 90 предприятий
2. Пищевая промышленность: 78 предприятий
3. Химическая промышленность: 60 предприятий
4. Целлюлозно-бумажная промышленность, издательская и полиграфическая деятельность: 59 предприятий
5. Машины и оборудование: 29 предприятий
